We want a class to validate that the grade received by a student on a homework is a percentage

In [1]:
class Homework(object):
    
    def __init__(self):
        self._grade = 0
        
    @property
    def grade(self):
        return self._grade
    
    @grade.setter
    def grade(self, value):
        if not (0 <= value <= 100):
            raise ValueError("Grade must be between 0 and 100")
        self._grade = value

In [2]:
galileo = Homework()
galileo.grade = 95

In [3]:
galileo.grade = 102

ValueError: Grade must be between 0 and 100

We also want to give the student a grade for an exam, where the exam has multiple subjects, each with a separate grade.

In [4]:
class Exam(object):
    
    def __init__(self):
        self._writing_grade = 0
        self._math_grade = 0
        
    @staticmethod
    def _check_grade(value):
        if not (0 <= value <= 100):
            raise ValueError("Grade must be between 0 and 100")

The problem is then that we need to reference this validation for each variable

In [5]:
class Exam(object):
    
    def __init__(self):
        self._writing_grade = 0
        self._math_grade = 0
        
    @staticmethod
    def _check_grade(value):
        if not (0 <= value <= 100):
            raise ValueError("Grade must be between 0 and 100")
            
    @property
    def writing_grade(self):
        return self._writing_grade
    
    @writing_grade.setter
    def writing_grade(self, value):
        self._check_grade(value)
        self._writing_grade = value
        
    @property
    def math_grade(self):
        return self._math_grade
    
    @math_grade.setter
    def math_grade(self, value):
        self._check_grade(value)
        self._math_grade = value

The best way to do this in python is to use a **descriptor**.

### Descriptor 

A **descriptor** class can provide *\_\_get\_\_* and *\_\_set\*\_* methods that let us reuse some validation function. 

In [6]:
class Grade(object):
    
    def __get__(*args, **kwargs):
        pass
    
    def __set__(*args, **kwargs):
        pass

In [7]:
class Exam(object):
    
    # class attributes
    math_grade = Grade()
    writing_grade = Grade()
    science_grade = Grade()

when we assign a property

```python
    exam = Exam()
    exam.writing_grade = 40
```

this will be interpreted as

```python
    Exam.__dict__['writing_grade'].__set__(exam, 40)
```

and when we retrieve a property

```python
    print(exam.writing_grade)
```

it will be interpreted as
```python
    print(Exam.__dict__['writing_grade'].__get__(exam, Exam)
```

In [10]:
class Grade(object):
    
    def __init__(self):
        self._value = 0
        
    def __get__(self, instance, instance_type):
        return self._value
    
    def __set__(self, instance, value):
        if not (0 <= value <= 100):
            raise ValueError('Grade must be between 0 and 100')
        self._value = value
        
class Exam(object):
    
    # class attributes
    math_grade = Grade()
    writing_grade = Grade()
    science_grade = Grade()

In [11]:
first_exam = Exam()
first_exam.writing_grade = 82
first_exam.science_grade = 99
print('Writing: ', first_exam.writing_grade)
print('Science: ', first_exam.science_grade)

Writing:  82
Science:  99


In [15]:
second_exam = Exam()
second_exam.writing_grade = 50
print('Writing (second_exam): ', second_exam.writing_grade, ' -> is right !')
print('Writing (first_exam): ', first_exam.writing_grade, ' -> is wrong !')

Writing (second_exam):  50  -> is right !
Writing (first_exam):  50  -> is wrong !


This is because both *first_exam* and *second_exam* share the same *Grade* instance. 

To solve this, we need the *Grade* class to keep track of its value for each unique *Exam* instance. 

In [17]:
import weakref

In [19]:
class Grade(object):
    
    def __init__(self):
        self._values = weakref.WeakKeyDictionary()
        
    def __get__(self, instance, instance_type):
        if instance is None: return self
        return self._values.get(instance, 0)
    
    def __set__(self, instance, value):
        if not (0 <= value <= 100):
            raise ValueError('Grade must be between 0 and 100')
        self._values[instance] = value

In [20]:
class Exam(object):
    
    # class attributes
    math_grade = Grade()
    writing_grade = Grade()
    science_grade = Grade()

In [21]:
first_exam = Exam()
first_exam.writing_grade = 82
second_exam = Exam()
second_exam.writing_grade = 99
print('First: ', first_exam.writing_grade, 'is right')
print('Second: ', second_exam.writing_grade, 'is right')

First:  82 is right
Second:  99 is right
